
Predicting if a person would buy life insurnace based on his age using logistic regression

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df.drop('bought_insurance' ,axis = 1),df.bought_insurance,test_size=0.2)

In [12]:
x_train['age'] = x_train['age']/100
x_test['age'] = x_test['age']/100

In [21]:
model = keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.fit(x_train,y_train,epochs=100)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 893ms/step - accuracy: 0.5000 - loss: 0.7019
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 0.5000 - loss: 0.7016
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.5000 - loss: 0.7012
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.5000 - loss: 0.7008
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.5000 - loss: 0.7005
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5000 - loss: 0.7001
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5000 - loss: 0.6997
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5000 - loss: 0.6994
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5000 - loss: 0.6990
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.5000 - loss: 0.6987
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.5000 - loss: 0.6983
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.5000 -

In [22]:
model.evaluate(x_test,y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - accuracy: 0.5000 - loss: 0.7218


[0.7217714190483093, 0.5]

In [26]:
model.predict(x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step


array([[0.73065406],
       [0.7429678 ],
       [0.7792063 ],
       [0.7632031 ],
       [0.7973668 ],
       [0.59914863]], dtype=float32)

In [27]:
y_test

,bought_insurance
20,0
11,0
24,1
22,1
8,1
6,0


In [29]:
coef, intercept = model.get_weights()
coef,intercept

(array([[0.9072523],
        [0.9044904]], dtype=float32),
 array([-0.09706972], dtype=float32))

In [30]:
def sigmoid(x):
  import math
  return 1/(1+math.exp(-x))

In [31]:
def prediction(age,affordibility):
  weight_sum = age*coef[0] + affordibility*coef[1] + intercept
  return sigmoid(weight_sum)

In [33]:
prediction(.47, 1)

<ipython-input-30-b240e5bc802f>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1/(1+math.exp(-x))


0.7744880814958632

Implementing gradient descent in plain python. The goal is to come up with same w1, w2 and bias that keras model calculated

In [36]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

In [37]:
def log_loss(y_true,y_predicted):
  epsilon = 1e-15
  y_predicted = [max(i,epsilon) for i in y_predicted]
  y_predicted = [min(i,1-epsilon) for i in y_predicted]
  y_predicted = np.array(y_predicted)
  return -np.mean(y_true*np.log(y_predicted)+(1-y_true)*np.log(1-y_predicted))

In [63]:
def gd(age,affordibility,y_true,epochs):
  w1=w2=1
  bias = 0
  learning_rate = 0.0005
  n = len(age)
  for i in range(epochs):
    weight_sum = w1*age + w2*affordibility+bias
    y_predicted = sigmoid(weight_sum)
    loss = log_loss(y_true,y_predicted)

    w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
    w2d = (1/n)*np.dot(np.transpose(affordibility),(y_predicted-y_true))

    bias_d = np.mean(y_predicted-y_true)

    w1= w1-learning_rate*w1d
    w2 = w2-learning_rate*w2d
    bias = bias-learning_rate*bias_d

    print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')



  return w1,w2,bias


In [72]:
gd(x_train['age'],x_train['affordibility'],y_train,850)

Epoch:0, w1:0.9999802003338587, w2:0.9999482880707311, bias:-0.00011189920249970335, loss:0.7019348212550554
Epoch:1, w1:0.9999604059308385, w2:0.9998965846780362, bias:-0.0002237846541539733, loss:0.7019036481755311
Epoch:2, w1:0.9999406167906284, w2:0.9998448898215916, bias:-0.00033565635586710004, loss:0.7018724834324511
Epoch:3, w1:0.9999208329129176, w2:0.9997932035010731, bias:-0.0004475143085437015, loss:0.7018413270241608
Epoch:4, w1:0.9999010542973947, w2:0.9997415257161566, bias:-0.0005593585130887232, loss:0.7018101789490054
Epoch:5, w1:0.9998812809437487, w2:0.9996898564665173, bias:-0.0006711889704074384, loss:0.70177903920533
Epoch:6, w1:0.9998615128516681, w2:0.9996381957518304, bias:-0.0007830056814054476, loss:0.7017479077914794
Epoch:7, w1:0.9998417500208414, w2:0.9995865435717707, bias:-0.0008948086469886786, loss:0.7017167847057992
Epoch:8, w1:0.9998219924509568, w2:0.9995348999260125, bias:-0.0010065978680633865, loss:0.7016856699466341
Epoch:9, w1:0.99980224014170

(np.float64(0.9850348228971378),
 np.float64(0.9590858986989911),
 np.float64(-0.09025150601392051))

In [45]:
coef,intercept

(array([[0.9072523],
        [0.9044904]], dtype=float32),
 array([-0.09706972], dtype=float32))